In [2]:
import torch

import matplotlib.pyplot as plt
import torchmetrics as tm

from config import *
from stages import *
from train import *
from data.ssa import SSA

from data.util import crop_q_between, split_weekdays_and_weekends

%reload_ext autoreload
%autoreload 2

In [8]:
mse = tm.MeanSquaredError().to(CONFIG.device)
mape = tm.MeanAbsolutePercentageError().to(CONFIG.device)
mat_q = CONFIG.load('mat_q.pt')
CONFIG.alpha = 1
CONFIG.device
mat_q

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  4.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  1., 16.,  ..., 29.,  0.,  0.],
        [ 0.,  0.,  9.,  ..., 16.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]], device='cuda:0')

##### Compare MSE and MAPE losses for different parameter configurations

Pre-train DBN and Train DBN attaching KELM on each epoch

In [26]:
results_wd = {}
results_we = {}

ssa = SSA(4, [2, 2], CONFIG.device)

CONFIG.spectral_threshold = 1440
CONFIG.dbn_hidden_layer_sizes = [80, 80, 80]

gamma = 1
reg_coeff = 1

SECTION_INDEX = 0

for gamma in [gamma]:#range(1, 26, 5):
  for reg_coeff in [reg_coeff]:#range(1, 102, 25):
    for P in range(3125 * 4, 25001, 3125):
        CONFIG.spectral_threshold = P
        for N in range(4, 324, 80):
          CONFIG.dbn_hidden_layer_sizes = [N, N, N]

          mat_q_trend, mat_q_resid = preprocess_data(CONFIG.spectral_threshold, mat_q)
          mat_c, mat_x, nonempty, representatives = compress_data(
              mat_q_resid.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
          # while mat_c.shape[1] == 1:
          #   CONFIG.alpha += 0.01
          #   print(f'alpha={CONFIG.alpha}')
          #   mat_c, mat_x, nonempty, representatives = compress_data(
          #     mat_q_resid.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
          mat_q_trend = mat_q_trend[:, nonempty][:, representatives].abs()
          mat_q_trend = mat_q_trend[:, SECTION_INDEX][:, None]
          mat_c = mat_c[:, SECTION_INDEX][:, None]

          (_, val_trend_wd_dataloader, _), (train_trend_we_dataloader, val_trend_we_dataloader, _) = crop_and_split_mat(mat_q_trend, CONFIG)
          # (_, val_resid_wd_dataloader, _), (_, val_resid_we_dataloader, _) = crop_and_split_mat(mat_q_resid, CONFIG)
          mat_c_wd_datasets, mat_c_we_datasets = split_mat(mat_c, CONFIG)
          print(gamma, reg_coeff, P, N, mat_c.shape)

          del mat_c

          dbn, kelm, val_resid_we_dataloader = train_with_config(CONFIG, mat_c_we_datasets, dbn_training_epochs=100, stride=1, gamma=gamma, reg_coeff=reg_coeff)

          mse_loss_trend = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_resid = torch.tensor([0.,]).to(CONFIG.device)
          mse_loss_overall = torch.tensor([0.,]).to(CONFIG.device)
          iter_trend = iter(val_trend_we_dataloader)
          n_samples = 0
          for X_resid, y_resid in val_resid_we_dataloader:
              X_trend, y_trend = next(iter_trend)
              pred_trend = ssa.forecast(X_trend.squeeze().T, 1).sum(0)[-1][None]
              mse_loss_trend += mse(pred_trend, y_trend)
              # print('TREND', y_trend, pred_trend)

              pred_resid = dbn(X_resid).squeeze()
              pred_resid = kelm(pred_resid).T
              mse_loss_resid += mse(pred_resid, y_resid)
              # print('RESID', y_resid, pred_resid)

              pred = pred_trend + pred_resid

              y = y_trend + y_resid
              # print('OVRLL', y, pred)
              # print('=====')

              mse_loss_overall += mse(pred, y)
              n_samples += 1

          mse_loss_trend /= n_samples
          mse_loss_resid /= n_samples
          mse_loss_overall /= n_samples

          print(f'gamma={gamma}, reg_coeff={reg_coeff}, WE P={CONFIG.spectral_threshold}, N={CONFIG.dbn_hidden_layer_sizes}, Loss_trend={mse_loss_trend.item()}, Loss_resid={mse_loss_resid.item()}, Loss_overall={mse_loss_overall.item()}')
          del dbn
          del kelm
        break

1 1 12500 4 torch.Size([2016, 1])
Epoch 0:
	RBM 0: Loss=-17.420974731445312
	RBM 1: Loss=0.6578211784362793
	RBM 2: Loss=0.22304177284240723
Epoch 5:
	RBM 0: Loss=-24.367263793945312
	RBM 1: Loss=0.029702186584472656
	RBM 2: Loss=-0.11160850524902344
Epoch 10:
	RBM 0: Loss=-31.273244857788086
	RBM 1: Loss=-0.43567895889282227
	RBM 2: Loss=-0.3647644519805908
Epoch 15:
	RBM 0: Loss=-38.08616256713867
	RBM 1: Loss=-0.7163071632385254
	RBM 2: Loss=-0.5539155006408691


  1%|          | 10/1000 [00:06<10:37,  1.55it/s]


KeyboardInterrupt: 